# Char-based Adder

The focus of this part of the project is on the **Transformer neural network architecture**, which was introduced in the (now famous) paper *Attention is All you Need* by [Vaswani et al. (2017)](https://arxiv.org/abs/1706.03762). This neural network architecture is THE foundation of all modern large language models (e.g. OpenAI's GPT line of models, Anthropic Claude, Google Gemini, Meta Llama, DeepSeek, etc.). You will train a transformer network on large amounts of text data. The transformer networks in this project adhere to the general structure of OpenAI's **Generative Pretrained Transformer (GPT)** models.

Once you create a database of aritmetic expressions in this notebook, you will train a small transformer to add numbers (i.e. evaluate strings of arithmetic operations, such as `'1+1='`).

In [ ]:
#!pip install matplotlib numpy  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use(['seaborn-v0_8-colorblind', 'seaborn-v0_8-darkgrid'])
plt.rcParams.update({'font.size': 20})

np.set_printoptions(suppress=True, precision=4)

# Automatically reload your external source code
%load_ext autoreload
%autoreload 2

## Task 1: Generate the addition dataset

In this task, you will write code to generate and preprocess a dataset composed of strings that represent arithmetic addition expressions involving up to 2 digit non-negative integer operands. For example, `'20+30=50'` and `'2+30=32'`are allowed, but not `'300+5=305'`. You will train a transformer on a large numbers of such expressions then you will prompt it to generate the answer to the right of the equals sign. For example, once trained you could prompt the transformer with `'21+23='` and it should return `'44'`.

#### Data format

In this project, we will be working with text data and we are implementing a **character-level model** (unlike the word-level model used in the Word Embedding project). This means that each data sample is a sequence of `T` characters (i.e. tokens), which, for example, could be a part of a sentence or the characters that make up an arithmetic expression. So all the data samples in the dataset will have shape `(N, T)`. We int-code each char/token in the dataset based on the character's position in the vocabulary.

In [ ]:
from addition_dataset import *

#### Verify `get_char2ind_map` and `make_addition_expressions` outputs

In the cell below, use the provided `get_char2ind_map` to assign the vocabulary dictionary to a variable called `char2ind_map`. 

To aid in interpretability of the int-coded tokens, we will represent the each `0-9` digit of the integer operands being summed as `0-9` in the int coding. We map the following chars to the next available ints in our coding scheme:
- `'+'` → 10
- `'='` → 11

We introduce a "fake" token `'.'` (int code: 12) within our vocabulary, our data samples, and our labels, which indicates the end of each addition expression (**end token**). For example `'47+51=98.'`. This helps the transformer know when the last "real" token/char in each addition expression has been reached (i.e. there are no more numbers to the right) and when it generates text after training, the transformer can output the int code corresponding `'.'` to signify that it is done generating text.

We introduce another "fake" token `'#'` (int code: 13), which we call the **padding token**. Our transformers must be trained on fixed-length sequences, but different addition expressions have different length (i.e. the length of `'1+1=2'` is shorter than `'1+9=10'`). To overcome this issue with samples in our dataset, we use the padding token to right-pad any expression that has fewer characters than our longest supported expression `'99+99=198'`.

In [ ]:


print("Character to index mapping:\n", char2ind_map)

When executed, the function should print:

```
Character to index mapping:
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '+': 10, '=': 11, '.': 12, '#': 13}
```


In the cell below, use the provided `make_addition_expressions` to generate 10,000 addition expressions involving at most 2 digit operands (i.e. maximum operand of `99`). Use the default random seed. Assign the list of addition expressions to a variable called `addition_ds`. 

When executed, the function should print:
```
First 5/10000 expressions:
  ['1', '+', '6', '8', '=', '6', '9', '.', '#', '#']
  ['5', '9', '+', '5', '=', '6', '4', '.', '#', '#']
  ['9', '0', '+', '2', '2', '=', '1', '1', '2', '.']
  ['2', '5', '+', '1', '8', '=', '4', '3', '.', '#']
  ['3', '3', '+', '1', '7', '=', '5', '0', '.', '#']
```

### Create addition int-coded dataset samples and labels

To do this we must
- convert each char in each addition expression into an int-code (*using the vocabulary*).
- define the "class labels" or the characters we want the transformer to predict at each time step. This is simply the int code of next character in each the expression to the right of the current one. For example, for `'9+2=11'` the first token in the data sample is `9` and the first class label is `10` (the int code for `'+'`).

Implement `make_addition_samples_and_labels` in `addition_dataset.py` to perform the above tasks then test your code below.

In [ ]:
x_int_test, y_int_test = make_addition_samples_and_labels(addition_ds, char2ind_map)

print('First few samples (encoded):')
for i in range(5):
    print(x_int_test[i], "=>", y_int_test[i])

The above should print:

```
First few samples (encoded):
[1, 10, 6, 8, 11, 6, 9, 12, 13] => [10, 6, 8, 11, 6, 9, 12, 13, 13]
[5, 9, 10, 5, 11, 6, 4, 12, 13] => [9, 10, 5, 11, 6, 4, 12, 13, 13]
[9, 0, 10, 2, 2, 11, 1, 1, 2] => [0, 10, 2, 2, 11, 1, 1, 2, 12]
[2, 5, 10, 1, 8, 11, 4, 3, 12] => [5, 10, 1, 8, 11, 4, 3, 12, 13]
[3, 3, 10, 1, 7, 11, 5, 0, 12] => [3, 10, 1, 7, 11, 5, 0, 12, 13]
```

### Create dictionary converting int-coded tokens back into chars

Your transformer will output ints — the int-coded representation of the predicted next char. For example, if the transformer outputs `'11'` that should be converted to `'='` for interpretability. 

Implement and test `make_ind2char_mapping` in `addition_dataset.py` to create the dictionary that will use the vocabulary to map int-coded representations of tokens back to chars.

In [ ]:
ind2char_map = make_ind2char_mapping(char2ind_map)

print('Here is your ind2char_map:\n', ind2char_map)
print('Here is your char2ind_map:\n', char2ind_map)

The above should print:
```
Here is your ind2char_map:
 {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '+', 11: '=', 12: '.', 13: '#'}
Here is your char2ind_map:
 {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '+': 10, '=': 11, '.': 12, '#': 13}
```

### Convert from int-coded tokens back to characters

Because you will prompt your transformer with string input (like a chatbot) and we would like to make sense of the transformer predictions, let's write a function (`convert_int2str`) that automates the process of taking int-coded samples back into human-readable characters then test it below.

In [ ]:
x_str_test = convert_int2str(x_int_test, ind2char_map)
y_str_test = convert_int2str(y_int_test, ind2char_map)

print('First few samples converted back to chars:')
for i in range(5):
    print(x_str_test[i], "=>", y_str_test[i])

The above cell should output:

```
First few samples converted back to chars:
['1', '+', '6', '8', '=', '6', '9', '.', '#'] => ['+', '6', '8', '=', '6', '9', '.', '#', '#']
['5', '9', '+', '5', '=', '6', '4', '.', '#'] => ['9', '+', '5', '=', '6', '4', '.', '#', '#']
['9', '0', '+', '2', '2', '=', '1', '1', '2'] => ['0', '+', '2', '2', '=', '1', '1', '2', '.']
['2', '5', '+', '1', '8', '=', '4', '3', '.'] => ['5', '+', '1', '8', '=', '4', '3', '.', '#']
['3', '3', '+', '1', '7', '=', '5', '0', '.'] => ['3', '+', '1', '7', '=', '5', '0', '.', '#']
```

### Create train-validation split

Implement `make_train_val_split` in `addition_dataset.py` to divide the dataset into training and validation split then run the code below to check your implementation.

In [ ]:
x_train_test, y_train_test, x_val_test, y_val_test = make_train_val_split(x_int_test, y_int_test)
print(f'The training samples shape is: {x_train_test.shape} and should be torch.Size([9000, 9]).')
print(f'The training labels shape is: {y_train_test.shape} and should be torch.Size([9000, 9]).')
print(f'The validation samples shape is: {x_val_test.shape} and should be torch.Size([1000, 9]).')
print(f'The validation labels shape is: {y_val_test.shape} and should be torch.Size([1000, 9]).')

### Create addition dataset prompts and expected output

After training the transformer on expressions such as `'1+1=2.####'`, you will prompt it with `1+1=` and we expect it to generate `'2.'` (the `'.'` indicates it is done generating text). Let's write the `split_sum_and_answer` function to automate the process of generating the prompts and expected outputs for samples in either the train or validation sets.

In [ ]:
lhs_lists, ans_lists = split_sum_and_answer(x_str_test)

print('First five training prompts and answers:')
for i in range(5):
    print(f'prompt: {lhs_lists[i]} | answer: {ans_lists[i]}')

The above cell should output:

```
First five training prompts and answers:
prompt: 1+68= | answer: 69.#
prompt: 59+5= | answer: 64.#
prompt: 90+22= | answer: 112
prompt: 25+18= | answer: 43.
prompt: 33+17= | answer: 50.
```

### Automate addition dataset preprocessing

Call the functions that you wrote to get and preprocess the addition dataset all in one function called `get_addition_dataset`.

In [ ]:
x_train_test, y_train_test, x_val_test, y_val_test = get_addition_dataset(N=100)
print(f'The shape of your training samples is {x_train_test.shape} and it should be torch.Size([90, 9]).')
print(f'The shape of your training labels is {y_train_test.shape} and it should be torch.Size([90, 9]).')
print(f'The shape of your val samples is {x_val_test.shape} and it should be torch.Size([10, 9]).')
print(f'The shape of your val labels is {y_val_test.shape} and it should be torch.Size([10, 9]).')

# We need int coded everything!
assert x_train_test.dtype == torch.long
assert y_train_test.dtype == torch.long
assert x_val_test.dtype == torch.long
assert y_val_test.dtype == torch.long

# Task 2: Train a GPT model

In [ ]:
import os
import multiprocessing

import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

from mingpt.model import GPT
from mingpt.trainer import Trainer
from mingpt.utils import set_seed, setup_logging, CfgNode as CN

# Set multiprocessing start method for Jupyter compatibility
multiprocessing.set_start_method('fork', force=True)

In [ ]:
def get_config():

    C = CN()

    # system
    C.system = CN()
    C.system.seed = 3407
    C.system.work_dir = './outputs/'

    # model
    C.model = GPT.get_default_config()
    C.model.model_type = 'gpt-nano'

    # trainer
    C.trainer = Trainer.get_default_config()
    C.trainer.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster

    return C

In [ ]:
# This class is used by the Trainer to get batches of data
class AdditionSubset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data
        
    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        # return x (input ids) and y (labels) as torch tensors
        return self.x_data[idx], self.y_data[idx]


class AdditionDataset(Dataset):   

    @staticmethod
    def get_default_config():
        C = CN()
        C.ndigit = 2
        return C

    def __init__(self):
        self.subsets = {}
        self.ndigit = self.get_default_config().ndigit

    def add_subset(self, name, x_data, y_data):
        self.subsets[name] = AdditionSubset(x_data, y_data)

    def get_subset(self, name):
        return self.subsets[name]
    
    def __str__(self):
        return f"subsets: {list(self.subsets.keys())}"
    
    def get_vocab_size(self):
        # 10 digits + '+', '=', '.', '#'
        return 14
    
    def get_block_size(self):
        # render = astr + '+' + bstr + '=' + cstr + '.'  -> length = 3*ndigit + 4
        # x will be render[:-1], so block_size = (3*ndigit + 4) - 1
        return 3*self.ndigit + 3

In [ ]:
# checking that the dataset works as expected
x_train, y_train, x_val, y_val = get_addition_dataset(N=100)

d = AdditionSubset(x_train, y_train)
for n, x in enumerate(d):
    print(x)
    if n > 4: break

In [ ]:
config = get_config()
setup_logging(config)
set_seed(config.system.seed)

# construct train and test datasets
char2ind_map = get_char2ind_map()
x_train, y_train, x_val, y_val = get_addition_dataset(N=10000)

dataset = AdditionDataset()
config.data = dataset.get_default_config()
dataset.add_subset('train', x_train, y_train)
dataset.add_subset('test', x_val, y_val)

# construct the model
config.model.vocab_size = dataset.get_vocab_size()
config.model.block_size = dataset.get_block_size()
model = GPT(config.model)

# construct the trainer object
config.trainer.max_iters = 5000
config.trainer.device = 'cuda' if torch.cuda.is_available() else 'cpu'
trainer = Trainer(config.trainer, model, dataset.get_subset('train') )

print(config)

In [ ]:
# helper function for the evaluation of a model
def get_left_part(x, val):
    ''' finds the position of '=' in the input sequence x and returns everything up to and including '=' '''
    indices = (x == val).nonzero(as_tuple=True)[1]
    return x[:, :indices[0]+1]

def get_right_part(x, val):
    indices = (x == val).nonzero(as_tuple=True)[1]
    return x[:, indices[0]+1:]

def eval_split(trainer, split, max_batches=None):
    assert split in ['train', 'test'], "split must be either 'train' or 'test'"
    subset = dataset.get_subset(split)
    ndigit = dataset.ndigit
    results = []
    mistakes_printed_already = 0
    loader = DataLoader(subset, batch_size=100, num_workers=0, drop_last=False, pin_memory=False, persistent_workers=False)
    for b, (x, y) in enumerate(loader):
        for i in range(x.size(0)):
            question = get_left_part(x[i:i+1], char2ind_map['=']) # Working with 2D tensors
            answer = get_right_part(y[i:i+1], char2ind_map['=']) # Working with 2D tensors
            # let the model sample the rest of the sequence
            result = model.generate(question, ndigit+2, do_sample=False) # using greedy argmax, not sampling
            answer2 = get_right_part(result, char2ind_map['=']) # Working with 2D tensors
            correct = (answer2[0, :ndigit+2] == answer[0, :ndigit+2]).all(dim=0).cpu()
            results.append(int(correct))
            if not correct and mistakes_printed_already < 5:
                mistakes_printed_already += 1
                print(f"Question: {question[0]}  GPT claims {answer2[0]} but ground truth is {answer[0 ]} ")
        if max_batches is not None and b+1 >= max_batches:
            break
    rt = torch.tensor(results, dtype=torch.float)
    print("%s final score: %d/%d = %.2f%% correct" % (split, rt.sum(), len(results), 100*rt.mean()))
    return rt.sum()

# iteration callback
top_score = 0
def batch_end_callback(trainer):
    global top_score

    if trainer.iter_num > 0 and (trainer.iter_num+1) % 100 == 0:
        print(f"iter {trainer.iter_num+1}: train loss {trainer.loss.item():.5f}; iter_dt {trainer.iter_dt * 1000:.2f}ms")

    if trainer.iter_num > 0 and (trainer.iter_num+1) % 1000 == 0:
        # evaluate both the train and test score
        train_max_batches = {1: None, 2: None, 3: 5}[config.data.ndigit] # if ndigit=2 we can afford the whole train set, ow no
        model.eval()
        with torch.no_grad():
            train_score = eval_split(trainer, 'train', max_batches=train_max_batches)
            test_score  = eval_split(trainer, 'test',  max_batches=None)
            score = train_score + test_score
            # save the model if this is the best score we've seen so far
            if score > top_score:
                top_score = score
                print(f"new top score of {score} achieved")
                #print(f"saving model with new top score of {score}")
                #ckpt_path = os.path.join(config.system.work_dir, "adder.model.pt")
                #torch.save(model.state_dict(), ckpt_path)
        # revert model to training mode
        model.train()

trainer.set_callback('on_batch_end', batch_end_callback)

In [ ]:
# run the optimization
trainer.run()

# Test the trained model with your prompts

In [ ]:
# Test the model on custom prompts
test_prompts = ["1+1=", "5+3=", "12+34=", "5+6=", "99+1=", "42+17=", "8+9="]

with torch.no_grad():
    for prompt_str in test_prompts:
        prompt_tokens = [char2ind_map[c] for c in prompt_str]
        prompt_tensor = torch.tensor([prompt_tokens])
        
        result = model.generate(prompt_tensor, max_new_tokens=4, do_sample=False)
        result_str = ''.join([ind2char_map[int(token)] for token in result[0].tolist()])
        
        print(f"Prompt: {prompt_str:10} | Model output: {result_str}")

Reference results
```
Prompt: 1+1=       | Model output: 1+1=2.##
Prompt: 5+3=       | Model output: 5+3=8.##
Prompt: 12+34=     | Model output: 12+34=46.#
Prompt: 5+6=       | Model output: 5+6=11.#
Prompt: 99+1=      | Model output: 99+1=100.
Prompt: 42+17=     | Model output: 42+17=69.#
Prompt: 8+9=       | Model output: 8+9=17.#
```